# Searching the Roman_DESC Sims

The web server at TODO_URL has indexes of the transient truth tables and the Roman images.  (Hopefully in the not-too-distant future it will have the Rubin images as well.)

Search it by hitting the URL followed by `/<command>`, where `<command>` is one of `findimages`, ROB TODO.  You can pass additional parameters by adding one or more "/<kw>=<val>" to the end of the URL (sparating each kw/val pair with a slash), or sending a dictionary in the POST data.  Examples of both of these are below.
    
## Finding Images
    
The URL for finding images is `/findimages`.  You can filter on the following keywords:
* `num` — the pointing number
* `pointing_ra` — the RA of the pointing itself (not of any individual SCA)
* `pointing_dec` — the Dec of the pointing itself
* `exptime` — the time of the exposure
* `mjd` — the MJD of the exposure
* `pa` — the position angle of the exposure
* `scanuma` — the number of the SCA within the exposure
* `ra` — the central RA of the SCA (this is not useful, because you would have to match the float value exactly)
* `dec` — the central Dec of the SCA (also not useful)
* `containing` — a (ra,dec) (in decimal degrees): find images that include this position on the sky (this _is_ useful)
    
Examples are below.


# Examples

The only python library you need is `requests`.  I also import `pandas` for convenience in showing the results of example searches.

Below, we set up the variable `server_url` to be used throughout the rest of the examples.  We also create a `requests` Session in `req` for use throughout the rest of the examples.

In many examples below, I check the `status_code` of the value returned by any requests to the webserver.  If this is not 200 (which is HTTP-speak for a normal return), it means something went wrong.  In that case, the `text` field of the return value may have useful information; if not to you, then potentially to give to Rob for debugging purposes.

If the `status_code` is 200, then all is well, and you can call the `json()` method on the return value to get a dictionary of values.  This dictionary is formatted so that it's suitable for immediate import into a pandas DataFrame (as you will see in the examples below).

In [41]:
import requests
import pandas

# Configure the server URL.  NOTE: right now this is a temporary development
#  server.  The actual URL once we're in production will be different.  So 
#  don't write this nasty url down anywhere.
server_url = 'https://webserver.romansnpit-roman-desc-simdex.development.svc.spin.nersc.org'
req = requests.Session()
req.verify = False        # Usually want this true, but if you're using a test server with a self-signed cert, need False

## Examples: Finding Images

### Examples 1: finding all images containing a point

In the first way, we'll encode the ra and dec in the URL itself.  (If we're still using the dev server, we'll get a `InsecureRequestWarning`; don't worry about that, it's just because the dev server doesn't have a proper SSL certificate.  The production one will.)

In [81]:
result = req.post( f'{server_url}/findimages/containing=(7.3152,-45.4391)' )
if result.status_code != 200:
    raise RuntimeError( f"Got status code {result.status_code}\n{result.text}" )

# If all went well (status_code was 200 and there was no exception), then you can pull
#  out the returned data with result.json()
df = pandas.DataFrame( result.json() )
print( f"Got {len(df)} images." )
display( df )
# Somebody somewhere seems to have sorted the columns alphabetically, which isn't really
#  what you would want, but oh well, they're all there

Got 7 images.


/global/common/software/nersc/pe/conda-envs/24.1.0/python-3.11/nersc-python/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'webserver.romansnpit-roman-desc-simdex.development.svc.spin.nersc.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


,boredec,borera,dec,dec_00,dec_01,dec_10,dec_11,exptime,filter,mjd,pa,pointingnum,ra,ra_00,ra_01,ra_10,ra_11,scanum
0,-45.6541,7.60523,-45.439134,-45.498363,-45.38106,-45.497173,-45.379864,161.025,R062,62000.02139,0.0,0,7.315209,7.226437,7.229427,7.401124,7.403751,15
1,-45.6229,7.63643,-45.407934,-45.467167,-45.34986,-45.465973,-45.348663,101.700,Z087,62000.22446,0.0,55,7.346569,7.257846,7.260835,7.432437,7.435062,15
2,-45.6541,7.60523,-45.439134,-45.498363,-45.38106,-45.497173,-45.379864,302.275,Y106,62000.39804,0.0,110,7.315209,7.226437,7.229427,7.401124,7.403751,15
3,-45.6229,7.63643,-45.407934,-45.467167,-45.34986,-45.465973,-45.348663,302.275,J129,62000.73412,0.0,165,7.346569,7.257846,7.260835,7.432437,7.435062,15
4,-45.6541,7.60523,-45.439134,-45.498363,-45.38106,-45.497173,-45.379864,302.275,H158,62001.07020,0.0,220,7.315209,7.226437,7.229427,7.401124,7.403751,15
5,-45.6229,7.63643,-45.407934,-45.467167,-45.34986,-45.465973,-45.348663,901.175,F184,62001.46174,0.0,275,7.346569,7.257846,7.260835,7.432437,7.435062,15
6,-45.6541,7.60523,-45.439134,-45.498363,-45.38106,-45.497173,-45.379864,901.175,K213,62002.33849,0.0,330,7.315209,7.226437,7.229427,7.401124,7.403751,15


If you prefer, instead of packing all of the search parameters in slash-separated keyword-value pairs in the URL, you can send POST data using the `json=` parameter:

In [85]:
result = req.post( f'{server_url}/findimages', json={ 'containing': [ 7.3152, -45.4391 ] } )
if result.status_code != 200:
    raise RuntimeError( f"Got status code {result.status_code}\n{result.text}" )
    
df = pandas.DataFrame( result.json() )
print( f"Got {len(df)} images" )
display( df )

Got 7 images


/global/common/software/nersc/pe/conda-envs/24.1.0/python-3.11/nersc-python/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'webserver.romansnpit-roman-desc-simdex.development.svc.spin.nersc.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


,boredec,borera,dec,dec_00,dec_01,dec_10,dec_11,exptime,filter,mjd,pa,pointingnum,ra,ra_00,ra_01,ra_10,ra_11,scanum
0,-45.6541,7.60523,-45.439134,-45.498363,-45.38106,-45.497173,-45.379864,161.025,R062,62000.02139,0.0,0,7.315209,7.226437,7.229427,7.401124,7.403751,15
1,-45.6229,7.63643,-45.407934,-45.467167,-45.34986,-45.465973,-45.348663,101.700,Z087,62000.22446,0.0,55,7.346569,7.257846,7.260835,7.432437,7.435062,15
2,-45.6541,7.60523,-45.439134,-45.498363,-45.38106,-45.497173,-45.379864,302.275,Y106,62000.39804,0.0,110,7.315209,7.226437,7.229427,7.401124,7.403751,15
3,-45.6229,7.63643,-45.407934,-45.467167,-45.34986,-45.465973,-45.348663,302.275,J129,62000.73412,0.0,165,7.346569,7.257846,7.260835,7.432437,7.435062,15
4,-45.6541,7.60523,-45.439134,-45.498363,-45.38106,-45.497173,-45.379864,302.275,H158,62001.07020,0.0,220,7.315209,7.226437,7.229427,7.401124,7.403751,15
5,-45.6229,7.63643,-45.407934,-45.467167,-45.34986,-45.465973,-45.348663,901.175,F184,62001.46174,0.0,275,7.346569,7.257846,7.260835,7.432437,7.435062,15
6,-45.6541,7.60523,-45.439134,-45.498363,-45.38106,-45.497173,-45.379864,901.175,K213,62002.33849,0.0,330,7.315209,7.226437,7.229427,7.401124,7.403751,15
